In [1]:
! pip install datasets evaluate transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=e4272fbe423db1981fbfd77bd15fbcb858fb21e92785d77190778dcdfb3dd7dd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install transformers --upgrade
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.15.5
    Uninstalling wandb-0.15.5:
      Successfully uninstalled wandb-0.15.5


In [3]:
import transformers

print(transformers.__version__)

4.31.0


In [4]:
model_checkpoint = "t5-base"    #"t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"
# model_checkpoint = "facebook/bart-base"

In [5]:
import pandas as pd
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Masking, LSTM
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
def load_dataset(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:
        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            paragraphs = ' '.join(example['targetParagraphs'])
            spoiler = example['spoiler'][0]
            data.append({'text': post_text + ' - ' + title + paragraphs, 'spoiler': spoiler})
    return pd.DataFrame(data)

def load_dataset_test(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:
        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            id = example['id']
            paragraphs = ' '.join(example['targetParagraphs'])
            # label = example['tags'][0] if 'tags' in example else None
            # if label in ['phrase', 'multi', 'passage']:
            data.append({'id': id, 'text': post_text + ' - ' + title + paragraphs})
    return pd.DataFrame(data)

test_data = load_dataset_test('/kaggle/input/mscitask1-spoiler-generation/test.jsonl')
train_data = load_dataset('/kaggle/input/mscitask1-spoiler-generation/train.jsonl')
validation_data = load_dataset('/kaggle/input/mscitask1-spoiler-generation/val.jsonl')
# test_data = load_dataset_test('/kaggle/input/clickbait-detection-msci641-s23/test.jsonl')
# all_datasets = pd.concat([test_data, train_data, validation_data])

In [7]:
print(type(train_data))
print(train_data[:2])
print(type(test_data))
print(test_data[:2])

<class 'pandas.core.frame.DataFrame'>
                                                text  \
0  Wes Welker Wanted Dinner With Tom Brady, But P...   
1  NASA sets date for full recovery of ozone hole...   

                               spoiler  
0  how about that morning we go throw?  
1                                 2070  
<class 'pandas.core.frame.DataFrame'>
   id                                               text
0   0  He Tackles A Nurse At The Hospital. Then You S...
1   1  Why you SHOULD be selfish at work - Why you SH...


In [8]:
train_data = np.array(train_data)
validation_data = np.array(validation_data)
test_data = np.array(test_data)

In [9]:
characters = ['!','"','#','$','%','&','(',')','*','+','/',':',';','<','=','>','@','^','`','|','~','\t','[',']','{','}','\\','.','-']
for i in range(len(train_data)):
    for j in characters:
        train_data[i][0] = train_data[i][0].replace(j,"")
        train_data[i][1] = train_data[i][1].replace(j,"")

for i in range(len(validation_data)):
    for j in characters:
        validation_data[i][0] = validation_data[i][0].replace(j,"")
        validation_data[i][1] = validation_data[i][1].replace(j,"")
        
for i in range(len(test_data)):
    for j in characters:
        test_data[i][1] = test_data[i][1].replace(j,"")

        
print(len(train_data))
print(len(validation_data))
print(len(test_data))

train_data = pd.DataFrame(train_data, columns=["text", "spoiler"])
validation_data = pd.DataFrame(validation_data, columns=["text", "spoiler"])
test_data = pd.DataFrame(test_data, columns=["id", "text"])

3200
400
400


In [10]:
train_data.to_csv('train_data.csv', index=False)
validation_data.to_csv('validation_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [11]:
from datasets import load_dataset
import datasets

dataset_file_train = '/kaggle/working/train_data.csv'
dataset_file_valid = '/kaggle/working/validation_data.csv'
data_file_test = '/kaggle/working/test_data.csv'

train_data = load_dataset('csv', data_files=dataset_file_train)
val_data = load_dataset('csv', data_files=dataset_file_valid)
test_data = load_dataset('csv', data_files=data_file_test)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d62f98377cb6b6c6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4c105ecc9eb6bcfe/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
print(train_data)
print(val_data)
print(test_data)

DatasetDict({
    train: Dataset({
        features: ['text', 'spoiler'],
        num_rows: 3200
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'spoiler'],
        num_rows: 400
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 400
    })
})


In [13]:
train_data["train"][0]

{'text': 'Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea  Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better IdeaIt’ll be just like old times this weekend for Tom Brady and Wes Welker Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium It seemed like a perfect opportunity for the two to catch up But Brady’s definition of catching up involves far more than just a meal In fact, it involves some literal catching as the Patriots quarterback looks to stay sharp during his fourgame Deflategate suspension I hit him up to do dinner Saturday night He’s like, ‘I’m going to be flying in from Ann Arbor later after the MichiganColorado football game, but how about that morning we go throw?’  Welker said on WQAM, per The Boston Globe And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah,

In [14]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(train_data["train"])

,text,spoiler
0,"Is 'Fargo' season three headed to St Cloud? UV Is 'Fargo' season three headed to St Cloud?MINNEAPOLIS — In its first two seasons, the television show Fargo brought its Midwestern crime tales to Bemidji, Duluth and Luverne, Minn It also hopscotched over to the Dakotas — North and South — racking up body counts in Fargo and Sioux Falls Now, season three is likely heading to St Cloud I think we're going to be in St Cloud this year, Noah Hawley, the show's creator, told the Aw Jeez podcast And we're going to do a little Eden Valley — for reasons that will become apparent later on St Cloud, in central Minnesota, has a population of approximately 60,000 Eden Valley, a small town of just 1,000, is located 45 minutes southwest of St Cloud Earlier, Hawley said the new season would be set in 2010 It was announced last month that Ewan McGregor would star, playing a pair of brothers It always starts for some reason with a catalytic moment, Hawley said of the show The first year was two guys in an emergency room The second year was a woman drives home with a guy sticking out of her windshield and makes dinner for her husband The third year revolves around two brothers The older one is very successful — the parking lot king of Minnesota — and the younger one is less successful as a parole officer There's sort of an original sin between them that blows up at the start of the story and sends things on their way, Hawley said This season will revolve around the two Ewans, but it will very quickly escalate to a lot of other things Hawley said that the Minnesota he presents in the show isn't meant to be the real Minnesota The job that I've been given is not to recreate Minnesota as it exists, but to recreate it as it existed in the movie Fargo, which I saw as respectful portrait with a certain comic aspect to it Despite its St Cloud setting, the third season will be shot up north in Calgary, Canada Shooting will likely begin in November, and the show will premiere next April I wish we could shoot here That would make me happy, Hawley said But it's America, so we have to make our television in Canada",season three is likely heading to St Cloud
1,"35 fucking infuriating things that happen in every romcom 35 Fucking Infuriating Things That Happen In Every RomComWe recently asked the BuzzFeed Community about the things that annoy them the most in romcom movies Here are some of the best results they submitted 1 Airport dash scenes Why do they always need to rush to the airport? Why not just call them? Or book a plane ticket? They're not gonna drop off the face of the Earth 2 Also, let's be honest, airport security is a joke in romcom world 3 Single people can apparently afford to have really nice apartments in big cities How is that possible? We can afford a box in a basement 4 And then there are the small details that just don't add up For example, in Leap Year they go to an outdoor wedding, in Ireland, in February And then to top it off they sleep on a bench after said wedding The average temperature in Ireland in February is 5°C 41°F 5 There's always a guy who is an absolute jerk who somehow becomes the main love interest He's a jerk No thank you 6 And what about the he's a fuckboy but there's something different about him clichè? Hmm? 7 Or a man who pursues a woman who isn't interested in him or even vehemently dislikes him Is this OK? IS IT??? 8 In fact, the normalisation of the stalking that characters do to win or win back the ones they want is pretty worrying when you think about it Is stalking OK? Let me think Nope 9 There's also always the message being sent out that guys are the beall and endall of EVERYTHING What is with all these women who have wonderful lives, Nora Jonesbeautiful kitchens, and supportive friends, but somehow they are deeply unhappy because they are single? SINGLE GOD NO There's more to life than getting a guy, btw 10 And what about the cheating? SO MUCH CHEATING And it's condoned and always results in ul

In [16]:
from datasets import load_metric
!pip install -U nltk
metric = load_metric("rouge")
metric_meteor = load_metric("meteor")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [17]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
print(metric.compute(predictions=fake_preds, references=fake_labels))
print(metric_meteor.compute(predictions=fake_preds, references=fake_labels))

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}
{'meteor': 0.9375}


In [18]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [21]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [22]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [23]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["spoiler"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_function_test(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    return model_inputs

In [24]:
# print(preprocess_function(train_data[:1]))
# len(preprocess_function(train_data[:1])[0]['input_ids'][0])
preprocess_function(train_data['train'][:2])

{'input_ids': [[21603, 10, 101, 7, 101, 40, 2304, 6834, 15, 26, 18428, 438, 3059, 24927, 6, 299, 20109, 7, 1593, 279, 10118, 10855, 22834, 101, 7, 101, 40, 2304, 6834, 15, 26, 18428, 438, 3059, 24927, 6, 299, 20109, 7, 1593, 279, 10118, 71, 10855, 22834, 196, 17, 22, 195, 36, 131, 114, 625, 648, 48, 1851, 21, 3059, 24927, 11, 101, 7, 101, 40, 2304, 101, 40, 2304, 5111, 1701, 1379, 30, 3, 9, 8327, 2252, 2478, 24, 3, 88, 3, 12655, 24927, 250, 3, 88, 22, 195, 36, 16, 1511, 21, 1771, 22, 7, 467, 344, 8, 368, 2789, 20109, 7, 11, 8327, 27676, 7, 44, 18584, 1954, 12750, 94, 3776, 114, 3, 9, 626, 1004, 21, 8, 192, 12, 3579, 95, 299, 24927, 22, 7, 4903, 13, 3, 11907, 95, 5806, 623, 72, 145, 131, 3, 9, 3506, 86, 685, 6, 34, 5806, 128, 26998, 3, 11907, 38, 8, 20109, 7, 20134, 1416, 12, 1049, 4816, 383, 112, 662, 7261, 374, 13710, 15, 5339, 9756, 27, 1560, 376, 95, 12, 103, 2634, 1856, 706, 216, 22, 7, 114, 6, 458, 196, 22, 51, 352, 12, 36, 7070, 16, 45, 6206, 27560, 865, 227, 8, 5847, 9939, 32, 1

In [25]:
preprocess_function_test(test_data['train'][:2])

{'input_ids': [[21603, 10, 216, 2067, 19376, 7, 71, 20252, 486, 37, 4457, 37, 29, 148, 1610, 363, 31, 7, 461, 37, 12324, 3021, 304, 37, 51, 17830, 20252, 11429, 7, 7309, 366, 978, 3, 17701, 6342, 15, 5405, 216, 1072, 6434, 5918, 71, 13824, 3186, 10555, 25, 317, 81, 39, 207, 803, 6, 186, 378, 369, 12, 809, 149, 28034, 280, 133, 36, 406, 135, 6, 149, 79, 143, 25, 8719, 11, 3993, 116, 25, 174, 34, 6, 11, 149, 79, 33, 132, 12, 380, 25, 16, 136, 1419, 148, 164, 237, 987, 909, 8, 822, 3, 318, 125, 133, 27, 103, 21, 135, 16, 1205, 58, 242, 997, 1201, 1490, 15146, 3038, 329, 1092, 152, 6, 8, 1525, 19, 650, 132, 19, 914, 150, 2006, 12, 125, 3, 88, 133, 103, 21, 112, 803, 264, 6, 116, 112, 23707, 19445, 12561, 172, 2381, 6, 3, 9, 1902, 1201, 1490, 10444, 44, 2474, 322, 432, 2788, 7, 16, 4361, 16990, 6, 332, 4, 6, 435, 2448, 16, 174, 13, 3, 9, 11546, 6, 3038, 329, 1092, 152, 737, 22, 17, 10888, 12, 253, 91, 3, 99, 3, 88, 228, 13213, 80, 13, 112, 282, 34, 2120, 91, 6, 3, 88, 47, 3, 9, 1588, 216, 1

In [26]:
# temp_var = preprocess_function(train_data)

In [27]:
print(type(train_data))


# train_data_tokenized = preprocess_function(train_data)
# test_data_tokenized = preprocess_function_test(test_data)
# validation_data_tokenized = preprocess_function(validation_data)

train_datasets = train_data.map(preprocess_function, batched=True)
validation_datasets = val_data.map(preprocess_function, batched=True)
test_datasets = test_data.map(preprocess_function_test, batched=True)

<class 'datasets.dataset_dict.DatasetDict'>


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
# validation_data_tokenized

In [29]:
# from datasets import Dataset, DatasetDict
# # train_data_tokenized = train_data_tokenized
# # validation_data_tokenized = validation_data_tokenized
# # test_data_tokenized = test_data_tokenized

# # Creating the DatasetDict
# train_dataset_dict = DatasetDict({
#     'train': train_data_tokenized
# })

# validation_dataset_dict = DatasetDict({
#     'validation': validation_data_tokenized
# })

# # Printing the dataset_dict
# # print(dataset_dict['train'])

In [30]:
# print(validation_dataset_dict['validation'])

In [31]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Load model directly
# from transformers import AutoModel
# model_longformer = AutoModel.from_pretrained("allenai/longformer-base-4096")

In [32]:
batch_size = 1    #for T5-small it is 16, for bart base, I am keeping it 4
gradient_accumulation_steps = 1  # Use gradient accumulation to effectively increase the batch size

model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,    #2e-5 or any value can be used here
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,  # Add gradient accumulation
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=25,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# data_collator_longformer = DataCollatorForSeq2Seq(tokenizer, model=model_longformer)

In [34]:
import nltk
import numpy as np

#     # Round numeric values in the "result" dictionary
#     rounded_result = {k: round(float(v), 4) if isinstance(v, (int, float)) else v for k, v in result.items()}
#     rounded_result_meteor = {k: round(float(v), 4) if isinstance(v, (int, float)) else v for k, v in result_meteor.items()}
    

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result_meteor = metric_meteor.compute(predictions=decoded_preds, references=decoded_labels)
    # Convert tuples to strings to avoid TypeError during rounding
    result = {key: str(value) if isinstance(value, tuple) else value for key, value in result.items()}
    result_meteor = {key: str(value) if isinstance(value, tuple) else value for key, value in result_meteor.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
#     # Round numeric values in the "result" dictionary
#     rounded_result = {k: round(float(v), 4) if isinstance(v, (int, float)) else v for k, v in result.items()}
#     rounded_result_meteor = {k: round(float(v), 4) if isinstance(v, (int, float)) else v for k, v in result_meteor.items()}
    
    result = {key: str(value) if isinstance(value, tuple) else value for key, value in result.items()}
    result_meteor = {key: str(value) if isinstance(value, tuple) else value for key, value in result_meteor.items()}
    
    merged_result = {**result, **result_meteor}
    
    merged_result = {key: str(value) if isinstance(value, tuple) else value for key, value in merged_result.items()}

    return merged_result
#     return rounded_result, rounded_result_meteor


In [35]:
def compute_metrics_meteor(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric_meteor.compute(predictions=decoded_preds, references=decoded_labels)

    # Convert tuples to strings to avoid TypeError during rounding
    result = {key: str(value) if isinstance(value, tuple) else value for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Round numeric values in the "result" dictionary
    rounded_result = {k: round(float(v), 4) if isinstance(v, (int, float)) else v for k, v in result.items()}

    return rounded_result


In [36]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_datasets['train'],
    eval_dataset=validation_datasets['train'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# trainer_longformer = Seq2SeqTrainer(
#     model_longformer,
#     args,
#     train_dataset=train_datasets['train'],
#     eval_dataset=validation_datasets['train'],
#     data_collator=data_collator_longformer,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics_meteor
# )

In [37]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,0.824200,0.693415,"AggregateScore(low=Score(precision=0.50374467516246, recall=0.3950312421041104, fmeasure=0.4009300615671617), mid=Score(precision=0.5508269325649844, recall=0.4377431468047417, fmeasure=0.4437955403157695), high=Score(precision=0.5924131508656787, recall=0.4814349023947423, fmeasure=0.48569636357309853))","AggregateScore(low=Score(precision=0.32415513882563507, recall=0.24116713583581448, fmeasure=0.24807727303755503), mid=Score(precision=0.37230533110680164, recall=0.284048367036263, fmeasure=0.2917552116519039), high=Score(precision=0.41967156942322376, recall=0.3283530998381211, fmeasure=0.33637622054894095))","AggregateScore(low=Score(precision=0.5002665993218384, recall=0.3908571259708004, fmeasure=0.39738353385688796), mid=Score(precision=0.5447898147604031, recall=0.43653778143686955, fmeasure=0.4417086861348908), high=Score(precision=0.5867395539713557, recall=0.4752899007117366, fmeasure=0.4805056434631726))","AggregateScore(low=Score(precision=0.5030752680683289, recall=0.3933248535347709, fmeasure=0.40005799078094334), mid=Score(precision=0.5446403479078116, recall=0.43432685693357936, fmeasure=0.44093277493439226), high=Score(precision=0.5903517606015308, recall=0.47887897416149655, fmeasure=0.48380662321544093))",7.647500,0.347145
2,0.622500,0.712881,"AggregateScore(low=Score(precision=0.527739741610514, recall=0.4370059980983654, fmeasure=0.4358954757256339), mid=Score(precision=0.5714736264552442, recall=0.4778611400925878, fmeasure=0.4768513234544952), high=Score(precision=0.6146321538245254, recall=0.5195119122954068, fmeasure=0.5177724887891918))","AggregateScore(low=Score(precision=0.3543141609768907, recall=0.28395698320495183, fmeasure=0.2851579799782941), mid=Score(precision=0.40616802345938374, recall=0.3275382621819256, fmeasure=0.3283892502617305), high=Score(precision=0.44978674108265104, recall=0.3698936635307192, fmeasure=0.36869021067127855))","AggregateScore(low=Score(precision=0.5185538893143213, recall=0.4258741552636579, fmeasure=0.4270935995543483), mid=Score(precision=0.5653810165528919, recall=0.4746477038655662, fmeasure=0.4734905364837827), high=Score(precision=0.6095405537171162, recall=0.5147671669739285, fmeasure=0.5153524643838899))","AggregateScore(low=Score(precision=0.520625008814715, recall=0.4304497631912279, fmeasure=0.43007567480851583), mid=Score(precision=0.5646860274121306, recall=0.47321055104758847, fmeasure=0.4724515723061629), high=Score(precision=0.6120335296647796, recall=0.5185080231578897, fmeasure=0.5178683569804874))",8.265000,0.385405
3,0.497100,0.777548,"AggregateScore(low=Score(precision=0.5331904431863072, recall=0.45087585738649055, fmeasure=0.45075347532399707), mid=Score(precision=0.5763436989889197, recall=0.4945579037620764, fmeasure=0.49023178450295496), high=Score(precision=0.6174235605999394, recall=0.5365393927766676, fmeasure=0.5297563986141037))","AggregateScore(low=Score(precision=0.36719628309292823, recall=0.30587960609493176, fmeasure=0.301557969220732), mid=Score(precision=0.4169130579387933, recall=0.35045207108716137, fmeasure=0.3457120067846823), high=Score(precision=0.4598829581774435, recall=0.3910793078305143, fmeasure=0.384376074059876))","AggregateScore(low=Score(precision=0.5267565052645476, recall=0.44491138252544865, fmeasure=0.4415243212615851), mid=Score(precision=0.5727273643431731, recall=0.4922134864432244, fmeasure=0.4880874401221894), high=Score(precision=0.6149689428422885, recall=0.532122136034154, fmeasure=0.5287386929330931))","AggregateScore(low=Score(precision=0.5285157490130379, recall=0.4517453973610428, fmeasure=0.4468037832252721), mid=Score(precision=0.5719504160055633, recall=0.49089313461939194, fmeasure=0.48674249271064685), high=Score(precision=0.6156832945700133, recall=0.534725525577376, fmeasure=0.5289380355243442))",8.787500,0.403219
4,0.375800,0.829872,"AggregateScore(low=Score(precision=0.5412672658377163, 

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is attempting to log a value of "AggregateScore(low=Score(precision=0.50374467516246, recall=0.3950312421041104, fmeasure=0.4009300615671617), mid=Score(precision=0.55082

TrainOutput(global_step=40000, training_loss=0.21655789289474486, metrics={'train_runtime': 24368.4952, 'train_samples_per_second': 3.283, 'train_steps_per_second': 1.641, 'total_flos': 7.049878619231232e+16, 'train_loss': 0.21655789289474486, 'epoch': 25.0})

In [38]:
print(test_data["train"][0])

{'id': 0, 'text': "He Tackles A Nurse At The Hospital Then You See What's On The Floor Next To Them  Male Nurse Breaks Down When His Friend Reveals He Can Give Him A KidneyWhen you think about your good friends, many things come to mind how bland life would be without them, how they make you laugh and smile when you need it, and how they are there to support you in any situation You may even ask yourself the question — what would I do for them in return? For 24yearold Graham McMillan, the answer is simple there is simply no limit to what he would do for his friends So, when his buddy Danny Kolzow, a 23yearold nurse at Baylor All Saints in Fort Worth, TX, found himself in need of a kidney, McMillan didn’t hesitate to find out if he could donate one of his As it turned out, he was a match He decided to surprise his friend with the good news, and did so in the most incredible way In this emotional clip, McMillan walks into the hospital where Kolzow works He has balloons and a sign in hand

In [39]:
import torch
# Function to generate spoilers using the trained model and tokenizer
def generate_spoilers(encoded_input, batch_size=1):
    with torch.no_grad():
        num_samples = len(encoded_input["input_ids"])
        decoded_spoilers = []

        for i in range(0, num_samples, batch_size):
            batch_input = {k: v[i:i+batch_size] for k, v in encoded_input.items()}
            batch_input = {k: torch.tensor(v).to(model.device) for k, v in batch_input.items()}
            output = model.generate(**batch_input)
            
            # Convert output to a list of tensors
            output = [torch.tensor(o).to(model.device) for o in output]

            # Perform batch decoding
            batch_decoded = tokenizer.batch_decode(output, skip_special_tokens=True)
            decoded_spoilers.extend(batch_decoded)

    return decoded_spoilers

# Generate spoilers for the test_data using the trained model
test_texts = test_data["train"]["text"]
encoded_input = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
batch_size = 4
decoded_test_preds = generate_spoilers(encoded_input, batch_size=batch_size)
# decoded_test_preds = generate_spoilers(encoded_input)

# Print the generated spoilers along with their corresponding ids
for spoiler, sample_id in zip(decoded_test_preds, test_data["train"]["id"]):
    print(f"id: {sample_id}, spoiler: {spoiler}")
    
# Create a DataFrame to store the spoilers and their IDs
spoilers_df = pd.DataFrame({"id": test_data["train"]["id"], "spoiler": decoded_test_preds})

# Strip leading and trailing whitespaces in the "spoiler" column
spoilers_df["spoiler"] = spoilers_df["spoiler"].str.strip()

# Check for any other representations of null spoilers (e.g., empty strings or NaN)
null_spoilers = spoilers_df[spoilers_df["spoiler"].isna() | (spoilers_df["spoiler"] == "")]

if not null_spoilers.empty:
    print("Null spoilers found. Replacing alternative representations...")
    print(null_spoilers)
    spoilers_df.replace({"": np.nan}, inplace=True)  # Replace empty strings with NaN
    spoilers_df["spoiler"].fillna("No Spoilers Generated", inplace=True)

    # Print rows with null spoilers
    print(spoilers_df[spoilers_df["spoiler"].isnull()])
    
# Save the DataFrame to a CSV file
spoilers_df.to_csv("generated_spoilers_t5_base_25.csv", index=False)

/tmp/ipykernel_28/2904365645.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input = {k: torch.tensor(v).to(model.device) for k, v in batch_input.items()}
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/tmp/ipykernel_28/2904365645.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = [torch.tensor(o).to(model.device) for o in output]


id: 0, spoiler: Danny Kolzow, a 23yearold nurse at Baylor All Saints in
id: 1, spoiler: Putting your needs before those of your colleagues is often seen as selfish behaviour But new research says
id: 2, spoiler: The big finding from Stanford economist Raj Chetty and his seven coauthors is not that rich
id: 3, spoiler: any of numerous wasps of the family Braconidae, the larva
id: 4, spoiler: remove any excess water from the yolks 1 Start by combining 1 cup salt and
id: 5, spoiler: John Williams
id: 6, spoiler: turning on your bed, just randomly checking the fridge for late night munchies and scroll
id: 7, spoiler: Flynn the dog does a lot to help out his 17yearold human, Hai
id: 8, spoiler: Lord Ivar Mountbatten has revealed that he is now dating a man after 
id: 9, spoiler: too much too soon Brandon says Focus on the power that you're getting along
id: 10, spoiler: I was with them The 1yearold must be changed, washed, 
id: 11, spoiler: Foursquare cofounder Dennis Crowley This Mistake W

In [40]:
### print the number of trainable parameters in the tranformers model
# from transformers import AutoModel, AutoTokenizer, MODEL_NAMES_MAPPING

# # Get a list of all available transformer model names
# all_model_names = list(MODEL_NAMES_MAPPING.keys())

# for model_name in all_model_names:
#     try:
#         # Load the model
#         model_class = AutoModel.from_pretrained(model_name)

#         # Count the number of trainable parameters
#         num_trainable_params = sum(p.numel() for p in model_class.parameters() if p.requires_grad)

#         print(f"Model: {model_name}")
#         print(f"Number of trainable parameters: {num_trainable_params}")
#         print("=" * 50)

#     except Exception as e:
#         print(f"Error loading model {model_name}: {str(e)}")
